In [53]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from pyquaternion import Quaternion
import numpy as np
import pandas as pd
import tqdm

In [16]:
import os

In [18]:
os.chdir("../")

In [83]:
# Loading the nuscene from local system
nusc = NuScenes(version='v1.0-mini', dataroot='data/input_data/v1.0-mini/', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.895 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [5]:
sensor = 'CAM_FRONT'
cam_front_data_token = my_sample['data'][sensor]
cam_front_data = nusc.get('sample_data', cam_front_data_token)
cam_front_data

{'token': 'e3d495d4ac534d54b321f50006683844',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'ego_pose_token': 'e3d495d4ac534d54b321f50006683844',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927612460,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
 'prev': '',
 'next': '68e8e98cf7b0487baa139df808641db7',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [87]:
camera_name = 'CAM_FRONT'

image_data_dict = dict()
# Iterate to scene
for scene in tqdm.tqdm(nusc.scene):
    #Iterate to sample
    token = scene['first_sample_token']
    while token != '':

        camera_token = nusc.get('sample',token)['data'][camera_name]
        annotation_list = sample['anns']

        filename, boxes, camera_intrinsic = nusc.get_sample_data(camera_token, selected_anntokens=annotation_list)
        if camera_token not in image_data_dict:
            image_data_dict[camera_token] = {'iID': camera_token,
                                                     'filePath': filename,
                                                     'annotations': []}
        for box in boxes:
            corners = view_points(box.corners(), np.array(camera_intrinsic), normalize=True)[:2, :]
            image_data_dict[camera_token]['annotations'].append({'name': box.name, 'corners': corners.tolist()})
            
        token = sample['next']
#         print(token)
#         break
        

100%|██████████| 10/10 [00:00<00:00, 189.28it/s]


## generate class distribution

In [69]:
with open('data/output_data/camera_front_boxes.json', 'r') as file:
    data = json.load(file)

In [70]:
iID_list = []
name_list = []
corners_list = []

In [71]:
for key, value in data.items():
    iID = value["iID"]
    annotations = value["annotations"]
    
    for annotation in annotations:
        name = annotation["name"]
        corners = annotation["corners"]
        
        # Append the data to the respective lists
        iID_list.append(iID)
        name_list.append(name)
        corners_list.append(corners)

# Create a Pandas DataFrame from the lists
df = pd.DataFrame({
    "iID": iID_list,
    "name": name_list,
    "corners": corners_list
})

In [72]:
df[['class_name', 'sub_category', 'grain']] = df['name'].str.split('.', expand=True, n=2)

In [82]:
class_counts = pd.DataFrame({'counts': df.class_name.value_counts(),
                                 'proportion': df.class_name.value_counts(normalize=True)
                                }).reset_index().rename(columns={"index":"class_name"})
class_counts['camera_name'] = 'camera_front'
class_counts = class_counts[['camera_name', 'class_name', 'counts', 'proportion']]
class_counts

,camera_name,class_name,counts,proportion
0,camera_front,vehicle,3050,0.568606
1,camera_front,movable_object,1144,0.213274
2,camera_front,human,1141,0.212714
3,camera_front,static_object,29,0.005406


In [81]:
sub_category_counts = pd.DataFrame({'counts': df[['class_name', 'sub_category']].value_counts(),
                                 'proportion': df[['class_name', 'sub_category']].value_counts(normalize=True)
                                }).reset_index()
sub_category_counts['camera_name'] = 'camera_front'
sub_category_counts = sub_category_counts[['camera_name', 'class_name', 'sub_category', 'counts', 'proportion']]
sub_category_counts

,camera_name,class_name,sub_category,counts,proportion
0,camera_front,vehicle,car,2349,0.437919
1,camera_front,human,pedestrian,1141,0.212714
2,camera_front,movable_object,trafficcone,594,0.110738
3,camera_front,movable_object,barrier,527,0.098248
4,camera_front,vehicle,bus,180,0.033557
5,camera_front,vehicle,truck,174,0.032438
6,camera_front,vehicle,motorcycle,147,0.027405
7,camera_front,vehicle,construction,101,0.018829
8,camera_front,vehicle,bicycle,82,0.015287
9,camera_front,static_object,bicycle_rack,29,0.005406
